In [1]:
import pandas as pd

# Load the data files (make sure to adjust paths if needed)
clearer_data = pd.read_excel("clearer_trade_data.xlsx")
internal_data = pd.read_excel("internal_trade_data.xlsx")


In [2]:
# Quick look at data to make sure it loaded correctly
print("Clearer Trade Data:")
print(clearer_data)
print("\nInternal Trade Data:")
print(internal_data)


Clearer Trade Data:
  delivery_month  quantity_mwh  price_eur_per_mwh
0        2025-03           100               30.0
1        2025-04           500               32.0
2        2025-05           550               32.5
3        2025-06           300               33.0

Internal Trade Data:
   trade_id     contract_type contract_period  quantity_mwh  price_eur_per_mwh
0         1          Outright         2025-03           100                 30
1         2          Outright         2025-04           200                 31
2         3          Outright         2025-05           250                 32
3         4  Outright Quarter            Q225           300                 33


In [3]:
# Identify the quarterly trade in internal data
quarterly = internal_data[internal_data["contract_type"] == "Outright Quarter"].iloc[0]

# Break down the quarterly trade (Q225 = April, May, June)
# Quarterly quantity and price split equally over 3 months
monthly_splits = [
    {"contract_type": "Quarterly Split", "contract_period": "2025-04", "quantity_mwh": quarterly["quantity_mwh"] / 3, "price_eur_per_mwh": quarterly["price_eur_per_mwh"]},
    {"contract_type": "Quarterly Split", "contract_period": "2025-05", "quantity_mwh": quarterly["quantity_mwh"] / 3, "price_eur_per_mwh": quarterly["price_eur_per_mwh"]},
    {"contract_type": "Quarterly Split", "contract_period": "2025-06", "quantity_mwh": quarterly["quantity_mwh"] / 3, "price_eur_per_mwh": quarterly["price_eur_per_mwh"]}
]

# Convert list to DataFrame and add to internal trades
split_df = pd.DataFrame(monthly_splits)
#internal_expanded = pd.concat([internal_data, split_df], ignore_index=True)

print("\nInternal Trades After Splitting Quarterly Contract:")
print(split_df)


Internal Trades After Splitting Quarterly Contract:
     contract_type contract_period  quantity_mwh  price_eur_per_mwh
0  Quarterly Split         2025-04         100.0                 33
1  Quarterly Split         2025-05         100.0                 33
2  Quarterly Split         2025-06         100.0                 33


In [23]:
#Calculate net quantities and weighted average prices per month

# Clean up any whitespace in column names
internal_expanded.columns = internal_expanded.columns.str.strip()

def calc_weighted_avg(group):
    total_qty = group['quantity_mwh'].sum()
    if total_qty == 0:
        return 0
    return (group['quantity_mwh'] * group['price_eur_per_mwh']).sum() / total_qty

# Sum quantities and compute weighted average prices grouped by month
qty_by_month = internal_expanded.groupby('contract_period')['quantity_mwh'].sum()
price_by_month = internal_expanded.groupby('contract_period').apply(calc_weighted_avg)

# Combine results into a single DataFrame
internal_summary = pd.DataFrame({
    'quantity_mwh': qty_by_month,
    'price_eur_per_mwh': price_by_month
}).reset_index().rename(columns={'contract_period': 'delivery_month'})

# Make sure all months of interest are included
months_of_interest = pd.DataFrame({'delivery_month': ['2025-03', '2025-04', '2025-05', '2025-06']})

internal_summary = months_of_interest.merge(internal_summary, on='delivery_month', how='left').fillna(0)

print("\nNetted Internal Positions (Monthly):")
print(internal_summary)


Netted Internal Positions (Monthly):
  delivery_month  quantity_mwh  price_eur_per_mwh
0        2025-03         100.0          30.000000
1        2025-04         300.0          31.666667
2        2025-05         350.0          32.285714
3        2025-06         100.0          33.000000


/var/folders/8f/py5g_g5x13bgvy8qp4dtr0c40000gn/T/ipykernel_75836/1766594769.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  price_by_month = internal_expanded.groupby('contract_period').apply(calc_weighted_avg)


In [24]:
# Compare internal data with clearer's monthly trades

# Merge clearer data and internal summary on delivery_month
reconciliation = pd.merge(
    clearer_data,
    internal_summary,
    on='delivery_month',
    how='outer',
    suffixes=('_clearer', '_internal')
)

# Calculate difference in quantity and price for reconciliation check
reconciliation['quantity_diff'] = reconciliation['quantity_mwh_internal'] - reconciliation['quantity_mwh_clearer']
reconciliation['price_diff'] = reconciliation['price_eur_per_mwh_internal'] - reconciliation['price_eur_per_mwh_clearer']

# Mark rows as 'Match' or 'Mismatch' based on differences
tolerance = 0.01  # small tolerance to avoid floating point issues
reconciliation['status'] = 'Match'
reconciliation.loc[
    (reconciliation['quantity_diff'].abs() > tolerance) | (reconciliation['price_diff'].abs() > tolerance),
    'status'
] = 'Mismatch'

print("\nFinal Reconciliation Table:")
print(reconciliation)


Final Reconciliation Table:
  delivery_month  quantity_mwh_clearer  price_eur_per_mwh_clearer  \
0        2025-03                   100                       30.0   
1        2025-04                   500                       32.0   
2        2025-05                   550                       32.5   
3        2025-06                   300                       33.0   

   quantity_mwh_internal  price_eur_per_mwh_internal  quantity_diff  \
0                  100.0                   30.000000            0.0   
1                  300.0                   31.666667         -200.0   
2                  350.0                   32.285714         -200.0   
3                  100.0                   33.000000         -200.0   

   price_diff    status  
0    0.000000     Match  
1   -0.333333  Mismatch  
2   -0.214286  Mismatch  
3    0.000000  Mismatch  
